# Object oriented error handling
- Error are defined by classes
- Error object is instantiated at error time
    - the class used tells you something about the problem
    - the error object may include specific information about the problem, like a file path that doesn't exist
- Java - compilier DEMANDS you handle all kinds of potential errors
- Python - hey whatever...
- but, if you get an error, and there's no handler, you CRASH
- [List of builtin errors]
(https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
- error types form a hierarchy(formed by using single inheritance)

# Iteration Protocol 
- signals end of iteration by throwing a StopIteration error

In [9]:
def fw(i):
    # iter gets the iterator for a sequence
    i2 = iter(i)
    # loop forever
    while True:
        e = next(i2)
        print(e)

In [10]:
# prints the list, then bombs with an error
# not handling the error that the iteration protocol
# throws when the iteration is exhausted

fw([1,2,3])

1
2
3


StopIteration: 

In [28]:
# have to 'catch' the 'StopIteration' error
# with 'try' 'except' statements
# errors other than 'StopIteration' will NOT be caught

def fw2(i):
    i2 = iter(i)
    while True:
        try:
            e = next(i2)
            print(e)
        # StopIteration is an error used to signal the
        # end of the iteration. catch it, and
        # break out of loop
        except StopIteration:
            break
    print('caught finish')
        
fw2([1,2,3])


1
2
3
caught finish


In [12]:
# ReverseIterList courtesy of Daniel Bauer

# another example of inheritance, from 'list'
# by implementing the iteration protocol,
# we make a list that iterates backwards

class ReverseIterList(list):
    
    # in this case the object itself is the iterator 
    def __iter__(self):
        # create an instance variable 'index', 
        # and set to the length of the list
        self.index = len(self)
        return(self)
    
    # calling the 'next' function on an object 
    # ultimately calls the '__next__' method on 
    # the object
    def __next__(self):
        # are we done?
        if self.index == 0:
            raise StopIteration
        else:
            # decrement index to go backwards
            self.index -= 1
        # return the list element that index selects
        return(self[self.index])

In [13]:
# looks like a normal list

rev = ReverseIterList(range(4))

print(rev)
print(rev[2])

[0, 1, 2, 3]
2


In [14]:
# for calls the __iter__ and __next__ methods on rev,
# and we iterate backwards

for x in rev:
    print(x)

3
2
1
0


In [15]:
# could this be improved?
# calling list with an arg of a list, 
# makes an unnessary copy of the list

def dotn(*pos):
    lists = [list(p) for p in pos]
    sum = 0
    for slice in zip(*lists):
        m = 1
        for s in slice:
            m *= s
        sum += m
    return sum    

def g(s, e):
    for j in range(s, e):
        yield j

dotn([5,3,9], g(10,12))

83

In [16]:
# what are the differences between dotn and dotn2
# can't call list on an infinite generator, so
# dotn can not be given an infinite generator
# also, calling list on a (finite) generator will
# make a list - but there is no need to make a list

# dotn2 works with infinite generators, and makes 
# no new lists

def dotn2(*pos):
    sum = 0
    iters = [iter(p) for p in pos]
    while True:
        try:
            m = 1
            for i in iters:
                m *= next(i)
            sum += m
        except StopIteration:
            return sum

# 'finally' clause is always executed, with or without error

In [17]:
# even tho the except clause does a break out of the loop, 
# the finally clause is still executed

def fw3(i):
    i2 = iter(i)
    while True:
        try:
            e = next(i2)
            print(str(e) + ' loop body')
        # StopIteration is not really an 'error', so just catch it and terminate
        except StopIteration as si:
            # si will be bound to the error object, but we don't need it
            print('before break')
            break
        finally:
            print(str(e) + ' finally clause')
        
# why is '3 finally clause' printed twice?
fw3([1,2,3])


1 loop body
1 finally clause
2 loop body
2 finally clause
3 loop body
3 finally clause
before break
3 finally clause


# 'with' statement
- very common pattern in software is to aquire some kind of resource or context,
use it for awhile, then return it or undo it. 
- common examples are file and network descriptors
    - very important to use descriptors correctly
    - running out of descriptors can crash a server

In [ ]:
# could write all this out...

path = '/tmp/no-such-file'

f = open(path, 'r')
# do things that might fail somehow
try:
    f.read()
finally: 
    # error or not, want to close the file descriptor
    # finally clause guarantees close will happen
    f.close()

In [ ]:
# instead, use 'with'
# less work, more consise

with open(path, 'r') as f:
    f.read()


# 'with' implements 'context manager' protocol
- like iteration protocol, a general protocol implemented by many classes
- depends on ```__enter__ and __exit__``` methods

In [ ]:
# 'with' use above roughly equivalent to:

f = open(path, 'r')
f.__enter__()
try:
    f.read()
finally:
    # always executed, closes the file descriptor
    f.__exit__()


# can explicitly 'raise'(or throw) an error 

In [29]:
def booboo(n):
    if n == 0:
        raise ValueError()
    return n

booboo(3)

3

In [30]:
booboo(0)

ValueError: 

# call stack frames display
- shows context of error
- most recent call last!
- very useful for debugging
- note: the most recent call frame is printed last

In [18]:
def foo(n):
    bar(1)

def bar(n):
    zap(2)

def zap(n):
    zip(3)
    
def zip(n):
    raise Exception()

foo(10)

Exception: 

# when an error is raised, Python will search the call stack for an error handler
- Python checks the current stack frame for a handler, then checks each
older frame in turn
- if no handler is found, the error is printed by the top level(and your program dies)

In [19]:
def tc3(f):
    try:
        return f()
    except ArithmeticError as ae:
        print('tc3: caught {}'.format(ae))    

def tc2(f):
    try:
        return tc3(f)
    # can catch any number of error types
    # in a single try
    except FileNotFoundError as fe:
        print('tc2: caught {}'.format(fe))
    except MemoryError as me:
        print('tc2: caught {}'.format(me))
    
def tc(b, f):
    if b:
        try:
            return tc2(f)
        # usually a good idea to catch
        # Exception at top level
        except Exception as e:
            print('tc: caught {}'.format(e))
    else:
        try:
            return tc2(f)
        except OSError as os:
            print('tc: caught {}'.format(os))
    
def noproblem():
    a = 5/4
    return a
    
def dbz2():
    try: 
        c = 1/0
        return c
    except ZeroDivisionError as z:
        print('dbz: caught {}'.format(z))
        
def dbz3():
    a = 1/0
    return a
    
def si():
    raise StopIteration
    
def fnf():
    # can put useful information about 
    # the error into the error object
    raise FileNotFoundError('/tmp/foo')

def me():
    raise MemoryError

In [20]:
# should run fine, and not generate an error

tc(1, noproblem)

1.25

In [21]:
#  caught the error it generated, inside dbz2

tc(1, dbz2)

dbz: caught division by zero


In [22]:
# no handler in dbz3
# caught by handler in tc3

tc(1, dbz3)

tc3: caught division by zero


In [23]:
# caught in 'tc' Exception handler

tc(1, si)

tc: caught 


In [24]:
# no handler on the call stack
# top level prints stack trace, 
# and your program crashes

tc(0, si)

StopIteration: 

# can define custom error object
- usually inherit from 'Exception'

In [25]:
class missedSecret(Exception):
    def __init__(self, secret, guess):
        self.secret = secret
        self.guess = guess
    
def guessSecret(guess):
    secret = 1234
    if guess != secret:
        raise missedSecret(secret, guess)
    return 'guess is correct'
    
guessSecret(1234)

'guess is correct'

In [26]:
guessSecret(34)

missedSecret: (1234, 34)

# can get error object and examine it
- info in error object might help decide how to handle the error

In [27]:
try:
    guessSecret(12)
except missedSecret as ms:
    # ms will be bound to the error object
    print('secret={} guess={}'.format(ms.secret, ms.guess))

secret=1234 guess=12
